# Solver For Cahn Hilliard's equation

This program solver the Cahn Hilliard equation for phase separation. It takes the initial distribution of fluid position and iterates due to diffusion.

It also attempts to couple the Cahn Hilliard equation and Navier Stokes equations, in order to obtain the displacement of a fluid due to external forces.

In [ ]:
#Setup FeniCs
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
pip install meshio

In [3]:
import meshio

In [ ]:
import random
from dolfin import *
from mshr import *
import matplotlib.pyplot as plt


# Class representing the initial conditions

#In this case, we are iterating two squares inside the mesh,
#to observe how they become circles, and how diffusion makes them merge into one,
#Ilustrating how phase separation works

class InitialConditions(UserExpression):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def eval(self, values, x):
        # Square 1
        center1 = (0.4, 0.5)
        side_length = 0.2
        half_side = side_length / 2.0
        in_square1 = (
            center1[0] - half_side < x[0] < center1[0] + half_side and
            center1[1] - half_side < x[1] < center1[1] + half_side
        )
        values[0] = 1.0 if in_square1 else 0.0

        # Square 2
        center2 = (0.6, 0.5)
        in_square2 = (
            center2[0] - half_side < x[0] < center2[0] + half_side and
            center2[1] - half_side < x[1] < center2[1] + half_side
        )
        values[0] += 1.0 if in_square2 else 0.0

        values[1] = 0.0

    def value_shape(self):
        return (2,)


# Class for interfacing with the Newton solver
class CahnHilliardEquation(NonlinearProblem):
    def __init__(self, a, L):
        NonlinearProblem.__init__(self)
        self.L = L
        self.a = a

    def F(self, b, x):
        assemble(self.L, tensor=b)

    def J(self, A, x):
        assemble(self.a, tensor=A)


# Model parameters
lmbda = 1.0e-02  # surface parameter
dt = 5.0e-06  # time step
theta = 0.5  # time stepping family, e.g. theta=1 -> backward Euler, theta=0.5 -> Crank-Nicolson

# Form compiler options
parameters["form_compiler"]["optimize"] = True
parameters["form_compiler"]["cpp_optimize"] = True

# Create mesh and build function space
mesh = UnitSquareMesh(100, 100)
P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
ME = FunctionSpace(mesh, P1 * P1) #Define Mixed finite elements

# Define trial and test functions
du = TrialFunction(ME)
q, v = TestFunctions(ME)

# Define functions
u = Function(ME)  # current solution
u0 = Function(ME)  # solution from the previous converged step

# Split mixed functions, taking the unknowns (c, mu)
dc, dmu = split(du)
c, mu = split(u)
c0, mu0 = split(u0)

# Create initial conditions and interpolate
u_init = InitialConditions(degree=1)
u.interpolate(u_init)
u0.interpolate(u_init)

# Compute the chemical potential df/dc
c = variable(c)
f = 100 * c**2 * (1 - c)**2 #Note that f can be whichever function that fullfils the parameters
dfdc = diff(f, c)

# Introduce an expression for mu_(n+theta)
mu_mid = (1.0 - theta) * mu0 + theta * mu

# Weak statement of the equations
L0 = c * q * dx - c0 * q * dx + dt * dot(grad(mu_mid), grad(q)) * dx
L1 = mu * v * dx - dfdc * v * dx - lmbda * dot(grad(c), grad(v)) * dx
L = L0 + L1

# Compute directional derivative about u in the direction of du (Jacobian)
a = derivative(L, u, du)

# Define boundary condition
def boundary_condition(x, on_boundary):
    return on_boundary

bc = [DirichletBC(ME.sub(0), Constant(0.0), boundary_condition)]

# Create nonlinear problem and Newton solver
problem = CahnHilliardEquation(a, L)
solver = NewtonSolver()
solver.parameters["linear_solver"] = "lu"
solver.parameters["convergence_criterion"] = "incremental"
solver.parameters["relative_tolerance"] = 1e-6

# Output file
file = File("output.pvd", "compressed")

# Step in time
t = 0.0
T = 50 * dt
i = 1
while t < T:
    t += dt
    u0.vector()[:] = u.vector()
    # Apply boundary condition to the problem
    for bci in bc:
        bci.apply(u.vector())
    solver.solve(problem, u.vector())
    # file << (u.split()[0], t)
    name = "img/output" + i * "a" + ".jpg"
    a = plot(u.split()[0], t, title="Cahn-Hilliard Trial round")
    plt.colorbar(a)
    plt.savefig(name)
    plt.clf()
    i = i + 1


In [ ]:
#This block of code creates a Gif of the phase separation. Use if needed.

from PIL import Image
import imageio
import os

# Define the path to your content folder
content_folder = '/content/'

# Get a list of image files in the content folder
image_files = [f for f in os.listdir(content_folder) if f.endswith('.jpg')]

# Sort the image files if needed (you can change the sorting criteria)
image_files.sort()ç

# Create a list to store the images
images = []

# Load each image and append it to the images list
for image_file in image_files:
    image_path = os.path.join(content_folder, image_file)
    img = Image.open(image_path)
    images.append(img)

# Define the path where you want to save the GIF
output_gif_path = '/content/output.gif'

# Save the images as a GIF
imageio.mimsave(output_gif_path, images, duration=1)  # Adjust duration as needed

# Display the generated GIF
from IPython.display import Image as IPImage, display
display(IPImage(output_gif_path))

In [ ]:
import mshr

The next blocks show the simulation for the couple of Cahn hilliard and Navier Stokes equations.

Note that the present case is attempting to simulate the fall of a fluid due to an external force (in this case, gravity).

In [ ]:
import meshio
msh2 = meshio.read("needle.xml")
meshio.write("needle.xml",msh2)
mesh = Mesh("needle.xml")

Warning: DOLFIN XML is a legacy format. Consider using XDMF instead.

In [ ]:
import random
#from fenics import *
from numpy.ma import tanh
import matplotlib.pyplot as plt

#################################################################################
#The initial condition takes the values with y > 0.9 and maps them to be water,
# Whilst nodes where y < 0.9 act as air.
class InitialConditions(UserExpression):
    def eval(self, values, x):
        x_ = x[0]
        y_ = x[1]
        random.seed(x_)
        rx = 2 * random.random() - 1
        values[0] = 0.5 * (1 + tanh(2 / wint * (y_ - L - 0.5e-4 * L * rx)))
        values[1] = 0.0

    def value_shape(self):
        return 2,

# Class for interfacing with the Newton solver
class CH_SOLVER(NonlinearProblem):
    def __init__(self, a, L):
        NonlinearProblem.__init__(self)
        self.L = L
        self.a = a

    def F(self, b, x):
        assemble(self.L, tensor=b)

    def J(self, A, x):
        assemble(self.a, tensor=A)

def left(c_):
    return (rho_eq_water - rho_int_water) / c_water * c_ + rho_int_water

def right(c_):
    return (rho_eq_air - rho_int_air) / (c_air - 1) * (c_ - c_air) + rho_eq_air

def rho(c_inp):
    return conditional(lt(c_inp, c_air), left(c_inp), right(c_inp))

def force(c_mu_):
    # Calculate density
    c__ = c_mu_.split(deepcopy=True)[0]
    den_ = rho(c__)
    # Calculate the buoyancy term (gravity acting on the water)
    force1 = -irho1 * ((den_ - rho_int_water) * g)
    # Calculate the capillary force (considered as zero for simplicity)
    force2 = Constant((0.0,0.0))
    # Calculate the total external force
    return force1 + force2

L = 0.9                  # Length of the domain
wint = L / 9             # Interface width (intentar con 9)
rho_int_water = 1000.0   # Initial density of water
rho_int_air = 1.0        # Initial density of air
rho_eq_water = 1000.0    # Equilibrium density of water
rho_eq_air = 1.0         # Equilibrium density of air
irho1 = Constant(1.0 / rho_int_water)  # Inverse of the initial density of water
c_water = 1.0            # Concentration of water
c_air = 0.0              # Concentration of air
nx = 200                 # Number of nodes in the x-direction
ny = 200                 # Number of nodes in the y-direction
Vm = 1e-5                # Molar volume
R = 8.31446              # Universal gas constant
T = 3000                 # Temperature
w_ref = 1e-9             # Reference width for mobility scaling
DU = 1.55E-12            # Diffusion coefficient for water
DO = 2.99e-9             # Diffusion coefficient for air
DZ = DU                  # Diffusion coefficient for additional species (if any)
DF = DU                  # Diffusion coefficient for additional species (if any)
M_ref = Vm / (R * T) * (DU + DZ + DO + DF)  # Reference mobility
M = wint / w_ref * M_ref  # Mobility
gama = 0.3854            # Surface tension coefficient
A0 = 12 * gama / ((c_water - c_air) ** 4 * wint)  # Parameter for capillarity
dt = 0.005               # Time step
idt = Constant(1.0 / dt)  # Inverse of the time step
kappa = 3 * wint * gama / (2 * (c_water - c_air) ** 2)  # Interface stiffness
nu = 10                  # Viscosity
g = Constant((0, -9.80665))  # Gravity vector, acts in the negative y-direction
theta = 0.5              # Time-stepping parameter


bcs = [] #Define Boundary conditions if needed

P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
P2 = VectorElement("Lagrange", mesh.ufl_cell(), 2)
elementCH = MixedElement([P1, P1])  # c mu
elementNS = MixedElement([P2, P1])  # u p (Taylor Hood elements)


MEch = FunctionSpace(mesh, elementCH)
MEns = FunctionSpace(mesh, elementNS)

# ch
d_c_mu = TrialFunction(MEch)
fai, pot = TestFunctions(MEch)
c_mu = Function(MEch)  # current solution
c_mu_0 = Function(MEch)  # solution from previous converged step

dc, dmu = split(d_c_mu)
c, mu = split(c_mu)
c0, mu0 = split(c_mu_0)

# Nvier Stokes with mixed elements
d_u_p = TrialFunction(MEns)
v, q = TestFunctions(MEns)
u_p = Function(MEns)  # current solution
u_p_0 = Function(MEns)  # solution from previous converged step

du, dp = split(d_u_p)
u, p = split(u_p)
u0, p0 = split(u_p_0)

# c mu Initial value
c_mu_init = InitialConditions(degree=1)
c_mu.interpolate(c_mu_init)
c_mu_0.interpolate(c_mu_init)
# u v The initial value defaults to 0
pvd_c = File("Concentration.pvd", "compressed")

# Compute the chemical potential df/dc
c = variable(c)
f = A0 * (c - c_air) ** 2 * (c - c_water) ** 2
df_dc = diff(f, c)

#Define the variational form for the Cahn Hilliard problem
L0 = c * fai * dx - \
     c0 * fai * dx + \
     dt * inner(inner(u0, grad(c)), fai) * dx + \
     dt * M * inner(grad(mu), grad(fai)) * dx
L1 = mu * pot * dx - df_dc * pot * dx - kappa * inner(grad(c), grad(pot)) * dx
L = L0 + L1

a = derivative(L, c_mu, d_c_mu)
problemCH = CH_SOLVER(a, L)
solverCH = NewtonSolver()
solverCH.parameters["linear_solver"] = "mumps"
solverCH.parameters["convergence_criterion"] = "incremental"
solverCH.parameters["relative_tolerance"] = 1e-6

# Define variational forms
F = (idt * inner(u - u0, v)
     + irho1 * Constant(theta) * nu * inner(nabla_grad(u), nabla_grad(v))
     + Constant(theta) * dot(dot(nabla_grad(u), u), v)
     + irho1 * Constant(1 - theta) * nu * inner(nabla_grad(u), nabla_grad(v))
     + Constant(1 - theta) * dot(dot(nabla_grad(u0), u0), v)
     - irho1 * p * div(v)
     - inner(force(c_mu), v)
     - q * div(u)
     ) * dx
J = derivative(F, u_p)

problemNS = NonlinearVariationalProblem(F, u_p, bcs, J)
solverNS = NonlinearVariationalSolver(problemNS)
solverNS.parameters['newton_solver']['linear_solver'] = 'mumps'

t = 0.0
n = 0
i = 1
Time = 0.3
while t <= Time:
    c_mu_0.vector()[:] = c_mu.vector()
    u_p_0.vector()[:] = u_p.vector()

    solverCH.solve(problemCH, c_mu.vector()) #Solve for Cahn Hilliard equation
    if n % 2 == 0 or n == 0:
        #pvd_c << c_mu.split()[0]

        vmin = 0.0
        vmax = 2.0
        name = "img3/output" + str(i) + ".jpg"
        a = plot(c_mu.split()[0], t, title="Cahn-Hilliard Trial round", vmin=vmin, vmax=vmax, colorbar=False)
        plt.colorbar(a, ticks=[vmin, vmax], label="Concentration")
        plt.savefig(name)
        plt.clf()

    solverNS.solve() #Update velocities, due to Navier Stokes equations

    t += dt
    n += 1
    i += 1
